This tutorial describes how to evaluate rules that are applicable to two consecutive periods (year and quarter).

In [ ]:
from arelle import ModelManager, Cntlr, ModelFormulaObject, ModelXbrl, ViewFileFormulae, XbrlConst, ViewFileRenderedGrid
from arelle import RenderingEvaluator 

In [ ]:
import pandas as pd
import numpy as np
from os import listdir
from os.path import join, isfile
import pickle
import re
from src import Evaluator
import logging
import data_patterns
import datetime

In [ ]:
DECIMALS = 0
RULES_PATH = join('..', 'solvency2-rules')
INSTANCES_DATA_PATH = join('..','data','instances')
DATAPOINTS_PATH = join('..', 'data', 'datapoints')

### Import rules

We start with importing the (t-1)-t rules that are applicable to two consecutive periods. We import a set of rules used to evaluate year data and a set of rules for quarter data.

#### S2_betweenperiods_ARS

In [ ]:
dfr_ARS = pd.read_excel(join(RULES_PATH,'S2_betweenperiods_ARS.xlsx'), engine='openpyxl')

#Capitalize row-column references:
column_replace = set([column for sublist in [row for row in dfr_ARS['pandas ex'].str.findall(r'c\d\d\d\d')] for column in sublist])
for ref in column_replace:
    dfr_ARS.replace(to_replace=ref, value=ref.capitalize(), inplace=True, regex=True)
column_replace = set([column for sublist in [row for row in dfr_ARS['pandas ex'].str.findall(r'r\d\d\d\d')] for column in sublist])
for ref in column_replace:
    dfr_ARS.replace(to_replace=ref, value=ref.capitalize(), inplace=True, regex=True)
dfr_ARS

#### S2_betweenperiods_QRS

In [ ]:
dfr_QRS = pd.read_excel(join(RULES_PATH,'S2_betweenperiods_QRS.xlsx'), engine='openpyxl')

#Capitalize row-column references:
column_replace = set([column for sublist in [row for row in dfr_QRS['pandas ex'].str.findall(r'c\d\d\d\d')] for column in sublist])
for ref in column_replace:
    dfr_QRS.replace(to_replace=ref, value=ref.capitalize(), inplace=True, regex=True)
column_replace = set([column for sublist in [row for row in dfr_QRS['pandas ex'].str.findall(r'r\d\d\d\d')] for column in sublist])
for ref in column_replace:
    dfr_QRS.replace(to_replace=ref, value=ref.capitalize(), inplace=True, regex=True)
dfr_QRS

### Import templates

Next we import the reporting data. We import the data of two consecutive periods. In the tutorial 'Convert XBRL-instances to CSV, HTML and pickles' the XBRL-instances are converted to pickle files per template. The pickle files are written to the data/instances folder. The rules are applicable to all tables with closed axis. We import these pickle files. When comparing two periods it can be the case that two different taxonomies are applicable. The right taxonomy has to be selected in the tutorial 'Convert XBRL-instances to CSV, HTML and pickles' to convert the XBRL-instance properly. 

The list _instances_ARS_ contains the names of the folders with the converted XBRL-instance for yearly data. The list _instances_QRS_ contains the names of the folders with the converted XBRL-instance for two consecutive quarters. Finally, we also have to define the category of the insurer. The rules are set-up for each type of insurer separately.

In [ ]:
instances_ARS = ["ars_260_instance", "ars_270_instance"]
instances_QRS = ["qrs_260_instance", "qrs_270_instance"]
categorie = 'Schade' #which type of insurer the instance belongs to (Schade, Herverzekeraar, Leven)

#### S2_betweenperiods_ARS

In [ ]:
df_datapoints = pd.read_csv(join(DATAPOINTS_PATH, 'ARS.csv'), sep=";").fillna("")  # load file to dataframe
dft = pd.DataFrame()
for instance in instances_ARS:
    df_closed_axis = pd.DataFrame()
    tables_closed_axis = []  # for listing all input tables with closed axis
    tables_complete_set = df_datapoints.tabelcode.sort_values().unique().tolist()  # list of all ARS tables
    tables = [table for table in tables_complete_set 
        if isfile(join(INSTANCES_DATA_PATH, instance, table + '.pickle'))]  # ARS tables found in the specified instance path
    for table in [table for table in tables if table not in ['S.14.01.01.04','S.30.03.01.01']]:  #tables:
        if isfile(join(INSTANCES_DATA_PATH, instance, table + '.pickle')):
            df = pd.read_pickle(join(INSTANCES_DATA_PATH,instance, table + '.pickle'))  # read dataframe
        else:
            continue   
        if df.index.nlevels > 2:  # if more than 2 indexes (entity, period), then the table has an open axis
            continue
        else:  # closed axis
            tables_closed_axis.append(table)  # add to relevant list
            # Add table to dataframe with all data from closed axis tables
            if len(df_closed_axis) == 0:  # no data yet --> copy dataframe
                df_closed_axis = df.copy()
            else:  # join to existing dataframe
                df_closed_axis = df_closed_axis.join(df)
    if len(dft) == 0:  # no data yet 
        dft = df_closed_axis
    else:  # join to existing dataframe
        dft=dft.append(df_closed_axis)
dft = dft.reset_index()
dft['categorie'] = categorie
numerical_columns = ['entity','period','categorie'] + [dft.columns[c] for c in range(len(dft.columns))
                        if ((dft.dtypes[c] == 'float64') or (dft.dtypes[c] == 'int64'))] #select only numerical columns
df_ARS = dft[numerical_columns]
df_ARS['period'] = df_ARS['period'].apply(lambda x: datetime.datetime.strptime(x,'%Y-%m-%d')) #convert to datetime
df_ARS.fillna(0,inplace=True)

In this tutorial we work with dummy data in order to show results

In [ ]:
# with open(join('..','tests','data','demo','ARS.pkl'), 'rb') as handle:
#     df_ARS = pickle.load(handle)
# df_ARS

#### S2_betweenperiods_QRS

In [ ]:
df_datapoints = pd.read_csv(join(DATAPOINTS_PATH, 'QRS.csv'), sep=";").fillna("")  # load file to dataframe
dft = pd.DataFrame()
for instance in instances_QRS:
    df_closed_axis = pd.DataFrame()
    tables_closed_axis = []  # for listing all input tables with closed axis
    # get tables
    tables_complete_set = df_datapoints.tabelcode.sort_values().unique().tolist()  # list of all QRS tables
    tables = [table for table in tables_complete_set 
        if isfile(join(INSTANCES_DATA_PATH,instance,table + '.pickle'))]  # QRS tables found in the specified INSTANCES_DATA_PATH
    for table in [table for table in tables if table not in ['S.14.01.01.04','S.30.03.01.01']]:  #tables:
        if isfile(join(INSTANCES_DATA_PATH,instance, table + '.pickle')):
            df = pd.read_pickle(join(INSTANCES_DATA_PATH,instance, table + '.pickle'))  # read dataframe
        else:
            continue
    
        if df.index.nlevels > 2:  # if more than 2 indexes (entity, period), then the table has an open axis
            continue
        else:  # closed axis
            tables_closed_axis.append(table)  # add to relevant list
        
            # Add table to dataframe with all data from closed axis tables
            if len(df_closed_axis) == 0:  # no data yet --> copy dataframe
                df_closed_axis = df.copy()
            else:  # join to existing dataframe
                df_closed_axis = df_closed_axis.join(df)
    if len(dft) == 0:  # no data yet 
        dft = df_closed_axis
    else:  # join to existing dataframe
        dft=dft.append(df_closed_axis)
dft=dft.reset_index()
dft['categorie']='Schade'
numerical_columns = ['entity','period','categorie'] + [dft.columns[c] for c in range(len(dft.columns))
                        if ((dft.dtypes[c] == 'float64') or (dft.dtypes[c] == 'int64'))]
df_QRS = dft[numerical_columns]
df_QRS['period']=df_QRS['period'].apply(lambda x: datetime.datetime.strptime(x,'%Y-%m-%d')) #convert to datetime
df_QRS.fillna(0,inplace=True)

In this tutorial we work with dummy data in order to show results

In [ ]:
# with open(join('..','tests','data','demo','QRS.pkl'), 'rb') as handle:
#     df_QRS = pickle.load(handle)
# df_QRS

### Evaluate rules

#### Evaluate S2_betweenperiods_ARS

In [ ]:
miner = data_patterns.PatternMiner(df_patterns=dfr_ARS)
miner.df_data = df_ARS
miner.metapatterns = {'cluster': 'categorie'}
miner.convert_to_time(['entity', 'categorie'], 'period')
miner.df_data = miner.df_data.reset_index()

results = miner.analyze()
results

#### S2_betweenperiods_QRS

In [ ]:
miner = data_patterns.PatternMiner(df_patterns=dfr_QRS)
miner.df_data = df_QRS
miner.metapatterns = {'cluster':'categorie'}
miner.convert_to_time(['entity', 'categorie'], 'period', set_year=False)
miner.df_data = miner.df_data.reset_index()

results = miner.analyze()
results